In [1]:
import pprint
import copy
import multiprocessing as mp
import itertools
import numpy as np
from itertools import compress

In [2]:
from dataclasses import dataclass

@dataclass
class Link:
    start_node: int 
    end_node: int
    number_of_modules: int
    module_cost: int
    link_module: int

@dataclass
class Demand:
    start_node: int
    end_node: int
    demand_volume: int
    number_of_paths: int
    paths: list
    
@dataclass
class Path:
    demand_path_id: int
    links: list
    
@dataclass
class Network:
    number_of_links: int
    links: list
    number_of_demands: int
    demands: list

In [3]:
def load_network(network_file):
    lines = []

    with open(network_file) as f:
        for line in f:
            lines.append(line[:-1])

    number_of_links = int(lines[0])

    links = []

    for i in range(1, number_of_links + 1):
        start_node, end_node, number_of_modules, module_cost, link_module = [int(item) for item in lines[i].split()]
        
        links.append(Link(start_node, end_node, number_of_modules, module_cost, link_module))

    number_of_demands = int(lines[number_of_links + 3])

    index_line = number_of_links + 5

    links_path = []
    demands = []

    for i in range(number_of_demands):
        paths = []

        start_node, end_node, demand_volume = [int(item) for item in lines[index_line].split()]
        
        number_of_paths = int(lines[index_line + 1])

        for j in range(index_line + 2, index_line + number_of_paths + 2):
            current_line = [int(item) for item in lines[j].split()]
            
            demand_path_id, links_path = current_line[0], current_line[1:]
            
            paths.append(Path(demand_path_id, links_path))

        demands.append(Demand(start_node, end_node, demand_volume, number_of_paths, paths))    

        index_line += number_of_paths + 3
    
    return Network(number_of_links, links, number_of_demands, demands)

In [4]:
network = load_network("net4.txt")

In [5]:
class Fitness():
    @staticmethod
    def fitnessDAP(solution):
#         print("Calculating fitness function DAP")
    
        flows_allocation = 0
        max_load = 0
    
        for link in range(1, network.number_of_links + 1):
            link_load = 0
            for demand in range(0, network.number_of_demands):
                for path in network.demands[demand].paths:
                    if link in path.links:
                        link_load += solution[demand][path.demand_path_id - 1]
            if link_load > max_load:
                max_load = link_load
    
#         print(max_load)
        return max_load  
        
    @staticmethod    
    def fitnessDDAP(solution):
#         print("Calculating fitness function DDAP")
    
        flows_allocation = 0
    
        for link in range(1, network.number_of_links + 1):
            link_load = 0
            for demand in range(0, network.number_of_demands):
                for path in network.demands[demand].paths:
                    if link in path.links:
                        link_load += solution[demand][path.demand_path_id - 1]
            link_size = link_load / network.links[link - 1].link_module
            flows_allocation += network.links[link - 1].module_cost * link_size
    
#         print(flows_allocation)
        return flows_allocation  

In [6]:
def subset_sum(numbers, target, partial=[], partial_sum=0):
    if partial_sum == target:
        yield partial
    if partial_sum >= target:
        return
    for i, n in enumerate(numbers):
        remaining = numbers[i + 1:]
        yield from subset_sum(remaining, target, partial + [n], partial_sum + n)

In [7]:
all_generated_solutions = []
possibility_values = []

for i in range(5):
    possibility_values.extend(list(range(30)))
    
for demand in network.demands:
    generated_solutions_for_demand = []
    demand_volume = demand.demand_volume
    
    for combination in subset_sum(possibility_values, demand_volume):
        if len(combination) == demand.number_of_paths and combination not in generated_solutions_for_demand:
            generated_solutions_for_demand.append(combination)
    
    all_generated_solutions.append(generated_solutions_for_demand)
    
# pprint.pprint(all_generated_solutions)
    

In [8]:
all_generated_solutions = []
possibility_values = []

def generate_values(demand, possibility_values):
    generated_solutions_for_demand = []
    demand_volume = demand.demand_volume
    
    for combination in subset_sum(possibility_values, demand_volume):
        if len(combination) == demand.number_of_paths and combination not in generated_solutions_for_demand:
            generated_solutions_for_demand.append(combination)
    
    return generated_solutions_for_demand

def permutate(demand_units):
    return [list(x) for x in set(itertools.permutations(demand_units))]

repeats = 6

for i in range(5):
    possibility_values.extend(list(range(repeats)))
    repeats -= 1

possibility_values.sort()
    
demands_list = []
for demand in network.demands:
    demands_list.append(demand)

# values generation
pool = mp.Pool(mp.cpu_count())
results = [pool.apply(generate_values, args=(demand, possibility_values.copy())) for demand in demands_list]
pool.close()

# permutation
pool = mp.Pool(mp.cpu_count())
results2 = [pool.map(permutate, i) for i in results]
pool.close()

results3 = []

for i in results2:
    temp = []
    for j in i:
        for k in j:
            temp.append(k)
    results3.append(temp)

results4 = (list(itertools.product(*results3)))

In [9]:
fitness_value = np.inf
demands = None

pool = mp.Pool(mp.cpu_count())
fitness_values = pool.map(Fitness.fitnessDDAP, results4)
pool.close()

minimum_value = min(fitness_values)
is_minimum_list = [y == minimum_value for y in fitness_values]
indices = list(compress(range(len(fitness_values)), is_minimum_list))
for index in indices:
    print(results4[index])

([3, 0, 0], [4, 0, 0], [0, 5], [2, 0, 0], [3, 0, 0], [4, 0, 0])
([3, 0, 0], [4, 0, 0], [5, 0], [2, 0, 0], [3, 0, 0], [4, 0, 0])
([3, 0, 0], [4, 0, 0], [4, 1], [2, 0, 0], [3, 0, 0], [4, 0, 0])
([3, 0, 0], [4, 0, 0], [1, 4], [2, 0, 0], [3, 0, 0], [4, 0, 0])
([3, 0, 0], [4, 0, 0], [3, 2], [2, 0, 0], [3, 0, 0], [4, 0, 0])
([3, 0, 0], [4, 0, 0], [2, 3], [2, 0, 0], [3, 0, 0], [4, 0, 0])
